In [1]:
# Öğrenci: Bekir Gümüş
# Numara:  222805010

In [2]:
# Video Sunum Linki:
# https://youtu.be/whJEFXbTvPs
# Not: Yorum satırları, uygulamanın daha açıklayıcı olması için yazılmıştır. Yazan kişi proje sahibidir.

In [3]:
!pip -q install -U pip setuptools wheel
!pip -q install -U --no-cache-dir numpy==1.26.4
!pip -q install -U --no-cache-dir \
  transformers \
  accelerate \
  datasets \
  peft \
  trl \
  bitsandbytes \
  sentence-transformers \
  faiss-cpu \
  fastapi \
  uvicorn \
  pyngrok \
  nest_asyncio \
  gradio


  # Bu hücre ile projede kullanılan RAG ve fine-tuning için gerekli kütüphaneleri kurar.


In [4]:
# Bu hücre duygulara ve müzik türlerine göre yapılandırılmıştır.
# Türkçe şarkılardan oluşan bu veri seti öneriler yapmak için oluşturulmuş gömülü bir JSON dosyasıdır.
# En altta bulunan özet bölümü ise 'categories' var mı? kaç kategori var? her kategoride kaç şarkı var? gibi bilgileri tutar ve yazdırır.

import json

music_kb = {
  "meta": {
    "language": "tr",
    "intended_year_range": "2022-2025",
    "note": "Elle oluşturulmuş mood/genre bilgi tabanı. Yıl doğrulaması gerekiyorsa sonradan temizleme adımı uygulanmalıdır."
  },
  "categories": [
    {
      "id": "pop_enerjik_1",
      "name": "Enerjik Türkçe Pop",
      "tags": ["pop", "enerjik", "mutlu", "hareketli"],
      "tracks": [
        {"title": "Antidepresan", "artist": "Mert Demir"},
        {"title": "Ateşe Düştüm", "artist": "Mert Demir"},
        {"title": "Ara", "artist": "Zeynep Bastık"},
        {"title": "Lan", "artist": "Zeynep Bastık"},
        {"title": "Yaz Gülü", "artist": "İrem Derici"},
        {"title": "Senin Yüzünden", "artist": "İrem Derici"},
        {"title": "Yansıma", "artist": "Hadise"},
        {"title": "Hayatım Kaymış", "artist": "Demet Akalın"},
        {"title": "Pırlanta", "artist": "Aleyna Tilki"},
        {"title": "Beni Bilen İyi Biliyor", "artist": "Oğuzhan Koç"},
        {"title": "Bi' Şehri Sevmek", "artist": "Mabel Matiz"},
        {"title": "Sana Yıldızları Ödediğimden", "artist": "Sıla"},
        {"title": "İlletim", "artist": "Mehmet Erdem"},
        {"title": "Deli Divane", "artist": "Edis"},
        {"title": "Üzülmedin mi?", "artist": "Simge"}
      ]
    },
    {
      "id": "pop_romantik_2",
      "name": "Romantik Türkçe Pop",
      "tags": ["pop", "romantik", "aşk", "soft"],
      "tracks": [
        {"title": "Olmazlara Vuruluyorum", "artist": "Mert Demir"},
        {"title": "Cehennemin Dibi", "artist": "Mert Demir"},
        {"title": "Dudak", "artist": "Mabel Matiz"},
        {"title": "İmkansız Bir Aşk Denir", "artist": "Melek Mosso"},
        {"title": "Beni Annem Bile Sevmedi", "artist": "Sıla"},
        {"title": "Kafi", "artist": "Derya Uluğ"},
        {"title": "Benim Ol", "artist": "Oğuzhan Koç"},
        {"title": "Dayanamam", "artist": "Zeynep Bastık"},
        {"title": "Kalbimi Kırdın", "artist": "İrem Derici"},
        {"title": "Aşktan Ne Haber", "artist": "Edis"},
        {"title": "Söyleme", "artist": "Hadise"},
        {"title": "Yalnızlık", "artist": "Aleyna Tilki"},
        {"title": "Aşkın Mevsimi", "artist": "Simge"},
        {"title": "Gözlerin", "artist": "Melek Mosso"},
        {"title": "Yoruldum", "artist": "Sıla"}
      ]
    },
    {
      "id": "pop_slow_3",
      "name": "Türkçe Slow Pop / Balad",
      "tags": ["pop", "slow", "duygusal", "balad"],
      "tracks": [
        {"title": "Canın Sağ Olsun", "artist": "Semicenk"},
        {"title": "Unutmak Öyle Kolay mı Sandın", "artist": "Semicenk"},
        {"title": "Pişman Değilim", "artist": "Semicenk"},
        {"title": "Mesafe", "artist": "Semicenk"},
        {"title": "Sen Kaldın", "artist": "Sıla"},
        {"title": "Çok Yazık", "artist": "Melek Mosso"},
        {"title": "Yandım", "artist": "Mabel Matiz"},
        {"title": "Ölüyorum Anla", "artist": "Mert Demir"},
        {"title": "Yanlış", "artist": "Oğuzhan Koç"},
        {"title": "Ağlama Ben Ağlarım", "artist": "İrem Derici"},
        {"title": "Bir Sebebi Var", "artist": "Simge"},
        {"title": "Boş Yere", "artist": "Hadise"},
        {"title": "Kördüğüm", "artist": "Aleyna Tilki"},
        {"title": "Kendime Yeni Bir Ben Lazım", "artist": "Derya Uluğ"},
        {"title": "İzmir Marşı Değil", "artist": "Sıla"}
      ]
    },
    {
      "id": "pop_akustik_4",
      "name": "Akustik / Soft Türkçe Pop",
      "tags": ["akustik", "soft", "sakin", "pop"],
      "tracks": [
        {"title": "Yine Sen", "artist": "Mabel Matiz"},
        {"title": "Bir Derdim Var", "artist": "Mert Demir"},
        {"title": "Yorgun", "artist": "Sıla"},
        {"title": "Gözlerine", "artist": "Melek Mosso"},
        {"title": "Sana Ne", "artist": "Oğuzhan Koç"},
        {"title": "Kırgın", "artist": "Derya Uluğ"},
        {"title": "Kül", "artist": "İrem Derici"},
        {"title": "Sakin", "artist": "Zeynep Bastık"},
        {"title": "İçimdeki Duman", "artist": "Edis"},
        {"title": "Yara", "artist": "Simge"},
        {"title": "Beni Bırakma", "artist": "Hadise"},
        {"title": "Bambaşka Biri", "artist": "Aleyna Tilki"},
        {"title": "Geceler", "artist": "Ezhel"},
        {"title": "Durma Yürüsene", "artist": "Melike Şahin"},
        {"title": "Diva Yorgun", "artist": "Melike Şahin"}
      ]
    },
    {
      "id": "pop_rap_feat_5",
      "name": "Pop x Rap (Feat / Karışık)",
      "tags": ["pop", "rap", "feat", "trend"],
      "tracks": [
        {"title": "Antidepresan", "artist": "Mert Demir"},
        {"title": "Nerdesin", "artist": "Sefo"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Affetmem", "artist": "BLOK3"},
        {"title": "Yana Yana", "artist": "Semicenk"},
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Sakin", "artist": "Ezhel"},
        {"title": "Geceler", "artist": "Ezhel"},
        {"title": "Arasan Da", "artist": "UZI"},
        {"title": "Paparazzi", "artist": "UZI"},
        {"title": "Kum Saati", "artist": "Motive"},
        {"title": "Yolunda", "artist": "Lvbel C5"},
        {"title": "Seni Düşündüm", "artist": "Reckol"},
        {"title": "Cıstak", "artist": "Batuflex"},
        {"title": "İmdat", "artist": "Madrigal"}
      ]
    },

    {
      "id": "arabesk_damar_6",
      "name": "Arabesk Damar (Ağır)",
      "tags": ["arabesk", "damar", "acı", "hüzün"],
      "tracks": [
        {"title": "Sevecek Sandım", "artist": "Semicenk"},
        {"title": "Kader Sağ Olsun", "artist": "Semicenk"},
        {"title": "Aldanırım", "artist": "Derya Bedavacı"},
        {"title": "Dert Olur", "artist": "Derya Bedavacı"},
        {"title": "Seni Böyle Sevdim", "artist": "Hakan Altun"},
        {"title": "Ağladım", "artist": "Hakan Altun"},
        {"title": "Senden Daha Güzel", "artist": "Ebru Yaşar"},
        {"title": "Affet", "artist": "Ebru Yaşar"},
        {"title": "Kırık Kalp", "artist": "Kubilay Karça"},
        {"title": "Gurur", "artist": "Kubilay Karça"},
        {"title": "Dön Desem", "artist": "Cem Adrian"},
        {"title": "Ben Seni Çok Sevdim", "artist": "Cem Adrian"},
        {"title": "Yar", "artist": "Ferhat Göçer"},
        {"title": "Yalan", "artist": "Ferhat Göçer"},
        {"title": "Yalnızlık", "artist": "Bergen"}
      ]
    },
    {
      "id": "arabesk_modern_7",
      "name": "Modern Arabesk / Pop Arabesk",
      "tags": ["arabesk", "modern", "pop arabesk"],
      "tracks": [
        {"title": "Dudaklarında Yalan", "artist": "Ebru Yaşar"},
        {"title": "Kehribar", "artist": "Ebru Yaşar"},
        {"title": "Dayan", "artist": "Hakan Altun"},
        {"title": "Sorma", "artist": "Hakan Altun"},
        {"title": "Yıkılsın", "artist": "Kubilay Karça"},
        {"title": "İçimdeki Sen", "artist": "Kubilay Karça"},
        {"title": "Yalancı Bahar", "artist": "Cem Adrian"},
        {"title": "Benim Hikayem", "artist": "Cem Adrian"},
        {"title": "Sende Kaldı Yüreğim", "artist": "Derya Bedavacı"},
        {"title": "Ölüyorum", "artist": "Derya Bedavacı"},
        {"title": "Yaralarım", "artist": "Sibel Can"},
        {"title": "Padişah", "artist": "Sibel Can"},
        {"title": "İhanetten Geri Kalan", "artist": "Hülya Avşar"},
        {"title": "Aşkın Kanunu", "artist": "Bülent Ersoy"},
        {"title": "Sevda Yüklü Kervanlar", "artist": "Yıldız Tilbe"}
      ]
    },
    {
      "id": "arabesk_rap_8",
      "name": "Arabesk Rap / Arabesk Trap",
      "tags": ["arabesk", "rap", "trap", "acı"],
      "tracks": [
        {"title": "Derman", "artist": "Gazapizm"},
        {"title": "Heyecanı Yok", "artist": "Gazapizm"},
        {"title": "Felaket", "artist": "Ezhel"},
        {"title": "Ağla", "artist": "Ezhel"},
        {"title": "Geceler", "artist": "Ezhel"},
        {"title": "Cennet", "artist": "Sefo"},
        {"title": "Affetmem", "artist": "BLOK3"},
        {"title": "Anılar", "artist": "BLOK3"},
        {"title": "Havhavhav", "artist": "Lvbel C5"},
        {"title": "Senden Güzeli Mi Var", "artist": "Lvbel C5"},
        {"title": "İntihar", "artist": "Motive"},
        {"title": "Bazen", "artist": "Motive"},
        {"title": "Sana Söz", "artist": "Reckol"},
        {"title": "Düştüm", "artist": "Reckol"},
        {"title": "Ayrılık", "artist": "Kubilay Karça"}
      ]
    },

    {
      "id": "rap_hype_9",
      "name": "Türkçe Rap Hype (Enerji)",
      "tags": ["rap", "enerjik", "hype", "trend"],
      "tracks": [
        {"title": "Affetmem", "artist": "BLOK3"},
        {"title": "Arasan Da", "artist": "UZI"},
        {"title": "Umrumda Değil", "artist": "UZI"},
        {"title": "Geceler", "artist": "Ezhel"},
        {"title": "Bilmem Mi?", "artist": "Sefo"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Nerdesin", "artist": "Sefo"},
        {"title": "Havhavhav", "artist": "Lvbel C5"},
        {"title": "Cıstak", "artist": "Batuflex"},
        {"title": "Dalga", "artist": "Batuflex"},
        {"title": "Aman Güzel Yavaş Yürü", "artist": "Ezhel"},
        {"title": "Şehrimin Tadı", "artist": "Ezhel"},
        {"title": "Kafamın İçi", "artist": "Motive"},
        {"title": "Beyaz", "artist": "Motive"},
        {"title": "Rampapapam", "artist": "Ben Fero"}
      ]
    },
    {
      "id": "rap_duygusal_10",
      "name": "Duygusal Türkçe Rap",
      "tags": ["rap", "duygusal", "hüzün", "gece"],
      "tracks": [
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Unutulacak Dünler", "artist": "Gazapizm"},
        {"title": "Sakin", "artist": "Ezhel"},
        {"title": "Geceler", "artist": "Ezhel"},
        {"title": "Kül", "artist": "Motive"},
        {"title": "Bazen", "artist": "Motive"},
        {"title": "Yara", "artist": "BLOK3"},
        {"title": "Anılar", "artist": "BLOK3"},
        {"title": "Konuştum", "artist": "Sefo"},
        {"title": "Kalp", "artist": "Sefo"},
        {"title": "Sana Söz", "artist": "Reckol"},
        {"title": "Düştüm", "artist": "Reckol"},
        {"title": "Gecenin Sonuna Yolculuk", "artist": "Sagopa Kajmer"},
        {"title": "Bir Pesimistin Gözyaşları", "artist": "Sagopa Kajmer"},
        {"title": "Suspus", "artist": "Ceza"}
      ]
    },
    {
      "id": "trap_trap_11",
      "name": "Türkçe Trap / T-Rap (Bass)",
      "tags": ["t-rap", "trap", "bass", "club"],
      "tracks": [
        {"title": "Havhavhav", "artist": "Lvbel C5"},
        {"title": "Cıstak", "artist": "Batuflex"},
        {"title": "Dalga", "artist": "Batuflex"},
        {"title": "Paparazzi", "artist": "UZI"},
        {"title": "Arasan Da", "artist": "UZI"},
        {"title": "Geceler", "artist": "UZI"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Bilmem Mi?", "artist": "Sefo"},
        {"title": "Rampapapam", "artist": "Ben Fero"},
        {"title": "Tanga", "artist": "Ben Fero"},
        {"title": "Güneşe Doğru", "artist": "Ezhel"},
        {"title": "Felaket", "artist": "Ezhel"},
        {"title": "Kafamın İçi", "artist": "Motive"},
        {"title": "Beyaz", "artist": "Motive"},
        {"title": "Seni Düşündüm", "artist": "Reckol"}
      ]
    },
    {
      "id": "drill_tr_12",
      "name": "Türkçe Drill (Sert)",
      "tags": ["drill", "sert", "agresif", "rap"],
      "tracks": [
        {"title": "Paparazzi", "artist": "UZI"},
        {"title": "Arasan Da", "artist": "UZI"},
        {"title": "Affetmem", "artist": "BLOK3"},
        {"title": "Cıstak", "artist": "Batuflex"},
        {"title": "Dalga", "artist": "Batuflex"},
        {"title": "Havhavhav", "artist": "Lvbel C5"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Bilmem Mi?", "artist": "Sefo"},
        {"title": "Susamam", "artist": "Şanışer"},
        {"title": "Sıkıntı Yok", "artist": "Contra"},
        {"title": "Geceler", "artist": "UZI"},
        {"title": "Rampapapam", "artist": "Ben Fero"},
        {"title": "Jagged", "artist": "Jagged"},
        {"title": "Leşker Asakir", "artist": "Ceza"},
        {"title": "Holocaust", "artist": "Ceza"}
      ]
    },
    {
      "id": "rap_gym_13",
      "name": "Spor / Motivasyon (Rap & Trap)",
      "tags": ["rap", "motivasyon", "gym", "enerji"],
      "tracks": [
        {"title": "Suspus", "artist": "Ceza"},
        {"title": "Holocaust", "artist": "Ceza"},
        {"title": "Yerinde Dur", "artist": "Ceza"},
        {"title": "Yenilmez", "artist": "Sagopa Kajmer"},
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Bilmem Mi?", "artist": "Sefo"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Affetmem", "artist": "BLOK3"},
        {"title": "Arasan Da", "artist": "UZI"},
        {"title": "Paparazzi", "artist": "UZI"},
        {"title": "Cıstak", "artist": "Batuflex"},
        {"title": "Dalga", "artist": "Batuflex"},
        {"title": "Rampapapam", "artist": "Ben Fero"},
        {"title": "Havhavhav", "artist": "Lvbel C5"},
        {"title": "Sıkıntı Yok", "artist": "Contra"}
      ]
    },
    {
      "id": "rap_story_14",
      "name": "Hikaye / Anlatı Rap",
      "tags": ["rap", "hikaye", "anlatı", "söz"],
      "tracks": [
        {"title": "Susamam", "artist": "Şanışer"},
        {"title": "Kırık Çocuk", "artist": "Sagopa Kajmer"},
        {"title": "Bir Pesimistin Gözyaşları", "artist": "Sagopa Kajmer"},
        {"title": "Gecenin Sonuna Yolculuk", "artist": "Sagopa Kajmer"},
        {"title": "Suspus", "artist": "Ceza"},
        {"title": "Holocaust", "artist": "Ceza"},
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Heyecanı Yok", "artist": "Gazapizm"},
        {"title": "Sakin", "artist": "Ezhel"},
        {"title": "Felaket", "artist": "Ezhel"},
        {"title": "Rampapapam", "artist": "Ben Fero"},
        {"title": "Yeniden", "artist": "Contra"},
        {"title": "Güzel Günler Göreceğiz", "artist": "Şanışer"},
        {"title": "İyi Bil", "artist": "Şehinşah"},
        {"title": "Nerdesin", "artist": "Sefo"}
      ]
    },

    {
      "id": "rock_enerjik_15",
      "name": "Enerjik Türkçe Rock",
      "tags": ["rock", "enerjik", "gitar", "canlı"],
      "tracks": [
        {"title": "Senden Daha Güzel", "artist": "Duman"},
        {"title": "Kufi", "artist": "Duman"},
        {"title": "Her Şeyi Yak", "artist": "Duman"},
        {"title": "Bir Derdim Var", "artist": "Mor ve Ötesi"},
        {"title": "Cambaz", "artist": "Mor ve Ötesi"},
        {"title": "Dursun Zaman", "artist": "maNga"},
        {"title": "Beni Benimle Bırak", "artist": "maNga"},
        {"title": "Holigan", "artist": "Athena"},
        {"title": "Senden, Benden, Bizden", "artist": "Athena"},
        {"title": "Yalnızlar Rıhtımı", "artist": "Adamlar"},
        {"title": "Rüyalarda Buruşmuşuz", "artist": "Büyük Ev Ablukada"},
        {"title": "Kendine İyi Bak", "artist": "Pinhani"},
        {"title": "Bir Kadın Çizeceksin", "artist": "Pinhani"},
        {"title": "Seni Dert Etmeler", "artist": "Madrigal"},
        {"title": "Dip", "artist": "Madrigal"}
      ]
    },
    {
      "id": "rock_duygusal_16",
      "name": "Duygusal Türkçe Rock",
      "tags": ["rock", "duygusal", "melankoli", "slow rock"],
      "tracks": [
        {"title": "Belki Üstümüzden Bir Kuş Geçer", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Ne Farkeder", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "İstanbul'da Sonbahar", "artist": "Teoman"},
        {"title": "Paramparça", "artist": "Teoman"},
        {"title": "Benimle Kayboldun", "artist": "Pinhani"},
        {"title": "Dön Bak Dünyaya", "artist": "Pinhani"},
        {"title": "Seni Dert Etmeler", "artist": "Madrigal"},
        {"title": "Geçme Artık Sokağımdan", "artist": "Madrigal"},
        {"title": "Sarmaşık", "artist": "Dolu Kadehi Ters Tut"},
        {"title": "Madem", "artist": "Dolu Kadehi Ters Tut"},
        {"title": "Rüzgar", "artist": "Adamlar"},
        {"title": "Düşünme", "artist": "Adamlar"},
        {"title": "Bir Derdim Var", "artist": "Mor ve Ötesi"},
        {"title": "Araf", "artist": "maNga"},
        {"title": "Gurur Benim Neyime", "artist": "Duman"}
      ]
    },
    {
      "id": "altrock_enerjik_17",
      "name": "Enerjik Alternatif Rock",
      "tags": ["alternatif rock", "enerjik", "indie rock"],
      "tracks": [
        {"title": "Yalnızlar Rıhtımı", "artist": "Adamlar"},
        {"title": "Rüyalarda Buruşmuşuz", "artist": "Büyük Ev Ablukada"},
        {"title": "Sarmaşık", "artist": "Dolu Kadehi Ters Tut"},
        {"title": "Madem", "artist": "Dolu Kadehi Ters Tut"},
        {"title": "Dip", "artist": "Madrigal"},
        {"title": "Geçme Artık Sokağımdan", "artist": "Madrigal"},
        {"title": "Güneye Giderken", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Boş Gemiler", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Geri Dönme", "artist": "Büyük Ev Ablukada"},
        {"title": "Benimle Kayboldun", "artist": "Pinhani"},
        {"title": "Yandım", "artist": "Jakuzi"},
        {"title": "Koca Bir Saçmalık", "artist": "Jakuzi"},
        {"title": "Gülüşünde Bir Şey Var", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Düşünme", "artist": "Adamlar"},
        {"title": "Seni Dert Etmeler", "artist": "Madrigal"}
      ]
    },
    {
      "id": "altrock_melankolik_18",
      "name": "Melankolik Alternatif Rock",
      "tags": ["alternatif rock", "melankolik", "gece"],
      "tracks": [
        {"title": "Belki Üstümüzden Bir Kuş Geçer", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Ne Farkeder", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Seni Dert Etmeler", "artist": "Madrigal"},
        {"title": "Dip", "artist": "Madrigal"},
        {"title": "Rüyalarda Buruşmuşuz", "artist": "Büyük Ev Ablukada"},
        {"title": "En Güzel Günüm Gecem", "artist": "Büyük Ev Ablukada"},
        {"title": "Dön Bak Dünyaya", "artist": "Pinhani"},
        {"title": "Kendine İyi Bak", "artist": "Pinhani"},
        {"title": "Yalnızlar Rıhtımı", "artist": "Adamlar"},
        {"title": "Rüzgar", "artist": "Adamlar"},
        {"title": "Sarmaşık", "artist": "Dolu Kadehi Ters Tut"},
        {"title": "Madem", "artist": "Dolu Kadehi Ters Tut"},
        {"title": "Yandım", "artist": "Jakuzi"},
        {"title": "Koca Bir Saçmalık", "artist": "Jakuzi"},
        {"title": "İstanbul'da Sonbahar", "artist": "Teoman"}
      ]
    },

    {
      "id": "indie_19",
      "name": "Türkçe Indie / Alternatif",
      "tags": ["indie", "alternatif", "cool", "şehir"],
      "tracks": [
        {"title": "Seni Dert Etmeler", "artist": "Madrigal"},
        {"title": "Dip", "artist": "Madrigal"},
        {"title": "Belki Üstümüzden Bir Kuş Geçer", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Ne Farkeder", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Rüyalarda Buruşmuşuz", "artist": "Büyük Ev Ablukada"},
        {"title": "Yandım", "artist": "Jakuzi"},
        {"title": "Koca Bir Saçmalık", "artist": "Jakuzi"},
        {"title": "Yalnızlar Rıhtımı", "artist": "Adamlar"},
        {"title": "Sarmaşık", "artist": "Dolu Kadehi Ters Tut"},
        {"title": "Madem", "artist": "Dolu Kadehi Ters Tut"},
        {"title": "Kendine İyi Bak", "artist": "Pinhani"},
        {"title": "Bir Kadın Çizeceksin", "artist": "Pinhani"},
        {"title": "Durma Yürüsene", "artist": "Melike Şahin"},
        {"title": "Diva Yorgun", "artist": "Melike Şahin"},
        {"title": "İmdat", "artist": "Madrigal"}
      ]
    },
    {
      "id": "alt_pop_20",
      "name": "Türkçe Alternatif Pop",
      "tags": ["alternatif pop", "indie pop", "modern"],
      "tracks": [
        {"title": "Durma Yürüsene", "artist": "Melike Şahin"},
        {"title": "Diva Yorgun", "artist": "Melike Şahin"},
        {"title": "Antidepresan", "artist": "Mert Demir"},
        {"title": "Ateşe Düştüm", "artist": "Mert Demir"},
        {"title": "Dudak", "artist": "Mabel Matiz"},
        {"title": "Yandım", "artist": "Mabel Matiz"},
        {"title": "Ara", "artist": "Zeynep Bastık"},
        {"title": "Lan", "artist": "Zeynep Bastık"},
        {"title": "Yansıma", "artist": "Hadise"},
        {"title": "Üzülmedin mi?", "artist": "Simge"},
        {"title": "Yaz Gülü", "artist": "İrem Derici"},
        {"title": "Kafi", "artist": "Derya Uluğ"},
        {"title": "İlletim", "artist": "Mehmet Erdem"},
        {"title": "Ben Seni Çok Sevdim", "artist": "Cem Adrian"},
        {"title": "İmkansız Bir Aşk Denir", "artist": "Melek Mosso"}
      ]
    },

    {
      "id": "gece_chill_21",
      "name": "Gece Chill (Sakin + Melankolik)",
      "tags": ["gece", "chill", "sakin", "melankoli"],
      "tracks": [
        {"title": "Sakin", "artist": "Ezhel"},
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Heyecanı Yok", "artist": "Gazapizm"},
        {"title": "Belki Üstümüzden Bir Kuş Geçer", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Ne Farkeder", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Rüyalarda Buruşmuşuz", "artist": "Büyük Ev Ablukada"},
        {"title": "Yandım", "artist": "Jakuzi"},
        {"title": "Seni Dert Etmeler", "artist": "Madrigal"},
        {"title": "Dip", "artist": "Madrigal"},
        {"title": "Canın Sağ Olsun", "artist": "Semicenk"},
        {"title": "Mesafe", "artist": "Semicenk"},
        {"title": "Olmazlara Vuruluyorum", "artist": "Mert Demir"},
        {"title": "Cehennemin Dibi", "artist": "Mert Demir"},
        {"title": "Dudak", "artist": "Mabel Matiz"},
        {"title": "Durma Yürüsene", "artist": "Melike Şahin"}
      ]
    },
    {
      "id": "lofi_sakin_22",
      "name": "Sakin / Düşük Enerji (Lo-fi hissi)",
      "tags": ["sakin", "lofi", "soft", "odak"],
      "tracks": [
        {"title": "Kendine İyi Bak", "artist": "Pinhani"},
        {"title": "Bir Kadın Çizeceksin", "artist": "Manga"},
        {"title": "İstanbul'da Sonbahar", "artist": "Teoman"},
        {"title": "Paramparça", "artist": "Teoman"},
        {"title": "İlletim", "artist": "Mehmet Erdem"},
        {"title": "Ben Seni Çok Sevdim", "artist": "Cem Adrian"},
        {"title": "Dön Desem", "artist": "Cem Adrian"},
        {"title": "Diva Yorgun", "artist": "Melike Şahin"},
        {"title": "Durma Yürüsene", "artist": "Melike Şahin"},
        {"title": "Canın Sağ Olsun", "artist": "Semicenk"},
        {"title": "Unutmak Öyle Kolay mı Sandın", "artist": "Semicenk"},
        {"title": "Pişman Değilim", "artist": "Semicenk"},
        {"title": "Yandım", "artist": "Mabel Matiz"},
        {"title": "Dudak", "artist": "Mabel Matiz"},
        {"title": "Yoruldum", "artist": "Sıla"}
      ]
    },

    {
      "id": "yaz_pop_23",
      "name": "Yazlık Türkçe (Neşeli / Dans)",
      "tags": ["yaz", "dans", "pozitif", "pop"],
      "tracks": [
        {"title": "Antidepresan", "artist": "Mert Demir"},
        {"title": "Ara", "artist": "Zeynep Bastık"},
        {"title": "Lan", "artist": "Zeynep Bastık"},
        {"title": "Yaz Gülü", "artist": "İrem Derici"},
        {"title": "Üzülmedin mi?", "artist": "Simge"},
        {"title": "Yansıma", "artist": "Hadise"},
        {"title": "Pırlanta", "artist": "Aleyna Tilki"},
        {"title": "Bilmem Mi?", "artist": "Sefo"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Affetmem", "artist": "BLOK3"},
        {"title": "Arasan Da", "artist": "UZI"},
        {"title": "Paparazzi", "artist": "UZI"},
        {"title": "Cıstak", "artist": "Batuflex"},
        {"title": "Dalga", "artist": "Batuflex"},
        {"title": "Holigan", "artist": "Athena"}
      ]
    },

    {
      "id": "kalp_kirikligi_24",
      "name": "Kalp Kırıklığı (Ayrılık / Hüzün)",
      "tags": ["ayrılık", "hüzün", "kalp kırıklığı", "duygusal"],
      "tracks": [
        {"title": "Canın Sağ Olsun", "artist": "Semicenk"},
        {"title": "Unutmak Öyle Kolay mı Sandın", "artist": "Semicenk"},
        {"title": "Mesafe", "artist": "Semicenk"},
        {"title": "Pişman Değilim", "artist": "Semicenk"},
        {"title": "Olmazlara Vuruluyorum", "artist": "Mert Demir"},
        {"title": "Cehennemin Dibi", "artist": "Mert Demir"},
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Sakin", "artist": "Ezhel"},
        {"title": "Ben Seni Çok Sevdim", "artist": "Cem Adrian"},
        {"title": "Dön Desem", "artist": "Cem Adrian"},
        {"title": "Aldanırım", "artist": "Derya Bedavacı"},
        {"title": "Dert Olur", "artist": "Derya Bedavacı"},
        {"title": "Seni Dert Etmeler", "artist": "Madrigal"},
        {"title": "Belki Üstümüzden Bir Kuş Geçer", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Kendine İyi Bak", "artist": "Pinhani"}
      ]
    },

    {
      "id": "ofke_isyan_25",
      "name": "Öfke / İsyan (Sert Mood)",
      "tags": ["öfke", "isyankar", "sert", "agresif"],
      "tracks": [
        {"title": "Holocaust", "artist": "Ceza"},
        {"title": "Suspus", "artist": "Ceza"},
        {"title": "Leşker Asakir", "artist": "Ceza"},
        {"title": "Sıkıntı Yok", "artist": "Contra"},
        {"title": "Susamam", "artist": "Şanışer"},
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Paparazzi", "artist": "UZI"},
        {"title": "Arasan Da", "artist": "UZI"},
        {"title": "Affetmem", "artist": "BLOK3"},
        {"title": "Cıstak", "artist": "Batuflex"},
        {"title": "Dalga", "artist": "Batuflex"},
        {"title": "Havhavhav", "artist": "Lvbel C5"},
        {"title": "Bilmem Mi?", "artist": "Sefo"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Her Şeyi Yak", "artist": "Duman"}
      ]
    },

    {
      "id": "umut_motivasyon_26",
      "name": "Umut / Motivasyon (Pozitif Akış)",
      "tags": ["umut", "motivasyon", "iyi his", "pozitif"],
      "tracks": [
        {"title": "Antidepresan", "artist": "Mert Demir"},
        {"title": "Ateşe Düştüm", "artist": "Mert Demir"},
        {"title": "Ara", "artist": "Zeynep Bastık"},
        {"title": "Lan", "artist": "Zeynep Bastık"},
        {"title": "Üzülmedin mi?", "artist": "Simge"},
        {"title": "Yaz Gülü", "artist": "İrem Derici"},
        {"title": "Yansıma", "artist": "Hadise"},
        {"title": "Kendime Yeni Bir Ben Lazım", "artist": "Derya Uluğ"},
        {"title": "Holigan", "artist": "Athena"},
        {"title": "Bir Derdim Var", "artist": "Mor ve Ötesi"},
        {"title": "Dursun Zaman", "artist": "maNga"},
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Bilmem Mi?", "artist": "Sefo"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Affetmem", "artist": "BLOK3"}
      ]
    },

    {
      "id": "gece_surus_trap_27",
      "name": "Gece Sürüşü (Trap/Rap Ağırlık)",
      "tags": ["gece sürüş", "trap", "rap", "akış"],
      "tracks": [
        {"title": "Arasan Da", "artist": "UZI"},
        {"title": "Paparazzi", "artist": "UZI"},
        {"title": "Geceler", "artist": "UZI"},
        {"title": "Affetmem", "artist": "BLOK3"},
        {"title": "Anılar", "artist": "BLOK3"},
        {"title": "Havhavhav", "artist": "Lvbel C5"},
        {"title": "Cıstak", "artist": "Batuflex"},
        {"title": "Dalga", "artist": "Batuflex"},
        {"title": "Sakin", "artist": "Ezhel"},
        {"title": "Felaket", "artist": "Ezhel"},
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Heyecanı Yok", "artist": "Gazapizm"},
        {"title": "Bilmem Mi?", "artist": "Sefo"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Rampapapam", "artist": "Ben Fero"}
      ]
    },

    {
      "id": "arabesk_kadin_28",
      "name": "Arabesk / Damar (Kadın Vokal)",
      "tags": ["arabesk", "kadın vokal", "damar", "hüzün"],
      "tracks": [
        {"title": "Aldanırım", "artist": "Derya Bedavacı"},
        {"title": "Dert Olur", "artist": "Derya Bedavacı"},
        {"title": "Yaralarım", "artist": "Sibel Can"},
        {"title": "Padişah", "artist": "Sibel Can"},
        {"title": "Sevda Yüklü Kervanlar", "artist": "Yıldız Tilbe"},
        {"title": "Vursalar Ölemem", "artist": "Yıldız Tilbe"},
        {"title": "Yalnızlık", "artist": "Bergen"},
        {"title": "Acıların Kadını", "artist": "Bergen"},
        {"title": "Affet", "artist": "Ebru Yaşar"},
        {"title": "Kehribar", "artist": "Ebru Yaşar"},
        {"title": "Senden Daha Güzel", "artist": "Ebru Yaşar"},
        {"title": "İmkansız Bir Aşk Denir", "artist": "Melek Mosso"},
        {"title": "Çok Yazık", "artist": "Melek Mosso"},
        {"title": "Yoruldum", "artist": "Sıla"},
        {"title": "Sen Kaldın", "artist": "Sıla"}
      ]
    },

    {
      "id": "arabesk_erkek_29",
      "name": "Arabesk / Damar (Erkek Vokal)",
      "tags": ["arabesk", "erkek vokal", "damar", "acı"],
      "tracks": [
        {"title": "Seni Böyle Sevdim", "artist": "Hakan Altun"},
        {"title": "Ağladım", "artist": "Hakan Altun"},
        {"title": "Dayan", "artist": "Hakan Altun"},
        {"title": "Sorma", "artist": "Hakan Altun"},
        {"title": "Ben Seni Çok Sevdim", "artist": "Cem Adrian"},
        {"title": "Dön Desem", "artist": "Cem Adrian"},
        {"title": "İlletim", "artist": "Mehmet Erdem"},
        {"title": "Yar", "artist": "Ferhat Göçer"},
        {"title": "Yalan", "artist": "Ferhat Göçer"},
        {"title": "Kırık Kalp", "artist": "Kubilay Karça"},
        {"title": "Gurur", "artist": "Kubilay Karça"},
        {"title": "Yıkılsın", "artist": "Kubilay Karça"},
        {"title": "Kader Sağ Olsun", "artist": "Semicenk"},
        {"title": "Sevecek Sandım", "artist": "Semicenk"},
        {"title": "Unutmak Öyle Kolay mı Sandın", "artist": "Semicenk"}
      ]
    },

    {
      "id": "trend_karma_30",
      "name": "Trend Karma (Pop/Rap/Rock Karışık)",
      "tags": ["trend", "karma", "pop", "rap", "rock"],
      "tracks": [
        {"title": "Antidepresan", "artist": "Mert Demir"},
        {"title": "Ateşe Düştüm", "artist": "Mert Demir"},
        {"title": "Canın Sağ Olsun", "artist": "Semicenk"},
        {"title": "Mesafe", "artist": "Semicenk"},
        {"title": "Affetmem", "artist": "BLOK3"},
        {"title": "Arasan Da", "artist": "UZI"},
        {"title": "Paparazzi", "artist": "UZI"},
        {"title": "Bilmem Mi?", "artist": "Sefo"},
        {"title": "Isabelle", "artist": "Sefo"},
        {"title": "Dua", "artist": "Gazapizm"},
        {"title": "Sakin", "artist": "Ezhel"},
        {"title": "Seni Dert Etmeler", "artist": "Madrigal"},
        {"title": "Belki Üstümüzden Bir Kuş Geçer", "artist": "Yüzyüzeyken Konuşuruz"},
        {"title": "Holigan", "artist": "Athena"},
        {"title": "Her Şeyi Yak", "artist": "Duman"}
      ]
    }
  ]
}

cats = None
if "music_kb" in globals() and isinstance(music_kb, dict) and "categories" in music_kb:
    cats = music_kb["categories"]
elif "categories" in globals():
    cats = categories
else:
    raise ValueError("Kategoriler bulunamadı. 'music_kb[\"categories\"]' veya 'categories' değişkeni olmalı.")

print("\n=== KATEGORİ / ŞARKI SAYISI ===\n")
print("Kategori sayısı:\n", len(cats))

total_tracks = 0
bad = []
for c in cats:
    name = c.get("name") or c.get("id") or "BilinmeyenKategori"
    n = len(c.get("tracks", []))
    total_tracks += n
    print(f"- {name}: {n} şarkı")
    if n != 15:
        bad.append((name, n))

print("\nToplam şarkı sayısı:", total_tracks)

if bad:
    print("\n⚠️ 15 şarkı olmayan kategoriler:")
    for name, n in bad:
        print(f"  - {name}: {n}")
else:
    print("\n Tüm kategoriler 15 şarkı içeriyor.")





=== KATEGORİ / ŞARKI SAYISI ===

Kategori sayısı:
 30
- Enerjik Türkçe Pop: 15 şarkı
- Romantik Türkçe Pop: 15 şarkı
- Türkçe Slow Pop / Balad: 15 şarkı
- Akustik / Soft Türkçe Pop: 15 şarkı
- Pop x Rap (Feat / Karışık): 15 şarkı
- Arabesk Damar (Ağır): 15 şarkı
- Modern Arabesk / Pop Arabesk: 15 şarkı
- Arabesk Rap / Arabesk Trap: 15 şarkı
- Türkçe Rap Hype (Enerji): 15 şarkı
- Duygusal Türkçe Rap: 15 şarkı
- Türkçe Trap / T-Rap (Bass): 15 şarkı
- Türkçe Drill (Sert): 15 şarkı
- Spor / Motivasyon (Rap & Trap): 15 şarkı
- Hikaye / Anlatı Rap: 15 şarkı
- Enerjik Türkçe Rock: 15 şarkı
- Duygusal Türkçe Rock: 15 şarkı
- Enerjik Alternatif Rock: 15 şarkı
- Melankolik Alternatif Rock: 15 şarkı
- Türkçe Indie / Alternatif: 15 şarkı
- Türkçe Alternatif Pop: 15 şarkı
- Gece Chill (Sakin + Melankolik): 15 şarkı
- Sakin / Düşük Enerji (Lo-fi hissi): 15 şarkı
- Yazlık Türkçe (Neşeli / Dans): 15 şarkı
- Kalp Kırıklığı (Ayrılık / Hüzün): 15 şarkı
- Öfke / İsyan (Sert Mood): 15 şarkı
- Umut / Motiv

In [5]:
# Bu hücre huggingface'den aldığımız model erişimini sağlar.
# Hücre çalıştırıldığında otomatik olarak bir textbox açılır ve token ister.
from huggingface_hub import login
import getpass

HF_TOKEN = getpass.getpass("HuggingFace token giriniz:  ")
login(token=HF_TOKEN)
print("HF login girişi yapılmıştır")


HuggingFace token giriniz:  ··········
HF login girişi yapılmıştır


In [6]:
# Bu hücre Mistral 7B Instruct dil modelini 4-bit quantization ile yükler.
# Amacımız büyük modeli düşük bellek kullanarak çalıştırmak ve chatbot için hazır hale getirmektir.

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"
)

print("✅ Model sorunsuz yüklendi")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✅ Model sorunsuz yüklendi


In [7]:
# Bu hücredeki sentence-transformers metni sayısal vektöre çevirir.
# Faiss ise şarkıları hızlı hızlı bulmamızı sağlar.
!pip -q install -U sentence-transformers faiss-cpu tqdm numpy


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.0 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.0 which is incompatible.


In [8]:
# Bu hücrede elimizdeki her bir şarkı doc haline gelir.
# Bu sayede yapay zekanın arama yapacağı metinleri bulmasına yardımcı oluyorum.
# En sonda şarkı + sanatçı + kategori = RAG dokümanı oluyor ve çıktıları topluyor.

import re, unicodedata

cats = None
if "music_kb" in globals() and isinstance(music_kb, dict) and "categories" in music_kb:
    cats = music_kb["categories"]
elif "categories" in globals() and isinstance(categories, list):
    cats = categories
else:
    raise ValueError("KB bulunamadı. 'categories = [...]' veya 'music_kb[\"categories\"]' olmalı.")

def clean(s: str) -> str:
    s = "" if s is None else str(s)
    s = unicodedata.normalize("NFKC", s).strip()
    s = re.sub(r"\s+", " ", s)
    return s

docs = []
metas = []

for ci, c in enumerate(cats):
    cat_name = clean(c.get("name", c.get("id", f"cat_{ci}")))
    tags = c.get("tags", [])
    if isinstance(tags, list):
        tags_str = ", ".join(clean(t) for t in tags)
    else:
        tags_str = clean(tags)

    for ti, t in enumerate(c.get("tracks", [])):
        title = clean(t.get("title", ""))
        artist = clean(t.get("artist", ""))
        if not title or not artist:
            continue

        doc = f"Kategori: {cat_name}\nEtiketler: {tags_str}\nŞarkı: {title}\nSanatçı: {artist}"
        docs.append(doc)
        metas.append({
            "title": title,
            "artist": artist,
            "cat": cat_name,
            "pos": f"{ci}:{ti}",
        })

print(" Doc sayısı:", len(docs))
print(" Örnek doc:\n", docs[0])


 Doc sayısı: 450
 Örnek doc:
 Kategori: Enerjik Türkçe Pop
Etiketler: pop, enerjik, mutlu, hareketli
Şarkı: Antidepresan
Sanatçı: Mert Demir


In [9]:
# Bu hücre, RAG için şarkı dokümanlarını vektöre çevirir
# ve faiss kullanarak hızlı benzerlik araması yapılacak index’i oluşturur.
# Böylece kullanıcı sorusuna en uygun şarkılar anlamsal olarak bulunabilir.

import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

EMB_MODEL = "intfloat/multilingual-e5-base"
emb = SentenceTransformer(EMB_MODEL)

def l2_normalize(x: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + 1e-12
    return x / n


passages = ["passage: " + d for d in docs]

vecs = []
bs = 64
for i in tqdm(range(0, len(passages), bs), desc="Embedding"):
    batch = passages[i:i+bs]
    v = emb.encode(batch, convert_to_numpy=True, show_progress_bar=False, normalize_embeddings=False)
    vecs.append(v)

vecs = np.vstack(vecs).astype("float32")
vecs = l2_normalize(vecs)

dim = vecs.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(vecs)

print(" FAISS hazır | dim:", dim, "| indexed:", index.ntotal)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Embedding: 100%|██████████| 8/8 [00:00<00:00,  9.11it/s]

 FAISS hazır | dim: 768 | indexed: 450


In [10]:
# Bu hücre çıktı olarak (benzerlik skoru, şarkı adı, sanatçı) listesi döndürür.

def retrieve(query: str, top_k: int = 12):
    q = "query: " + query
    qv = emb.encode([q], convert_to_numpy=True, normalize_embeddings=False).astype("float32")
    qv = l2_normalize(qv)

    scores, idxs = index.search(qv, top_k)

    out = []
    for s, idx in zip(scores[0], idxs[0]):
        if idx == -1:
            continue
        m = metas[idx]
        title = m.get("title", "")
        artist = m.get("artist", "")
        if not title or not artist:
            continue
        out.append((float(s), title, artist))

    return out


# Bu hücredeki fonksiyon bulunan şarkılar içinden tekrarları temizler
# LLM’e verilecek sade bir bağlam (context) metni oluşturur.
# Amaç: modele sadece gerekli şarkı isimlerini vermektir.

def build_context(retrieved, pool_n: int = 18):

    seen = set()
    lines = []
    for _, title, artist in retrieved:
        key = (title.lower().strip(), artist.lower().strip())
        if key in seen:
            continue
        seen.add(key)
        lines.append(f"{title} — {artist}")
        if len(lines) >= pool_n:
            break
    return "\n".join(lines)


In [11]:
# Bu hücre kullanıcının yazdığı cümlenin niyetini (intent) belirler.
# Amaç: müzik isteği mi, günlük sohbet mi, yoksa alakasız bir soru mu ayırt etmek.
# Böylece model her soruya şarkı önermeye çalışmaz.

import re

def classify_intent(q: str) -> str:
    qn = norm_tr(q)

    if any(x in qn for x in ["sarki", "muzik", "liste", "cal", "oner"]):
        return "MUSIC"


    if infer_style(q) != "genel":
        return "MUSIC"


    if any(x in qn for x in ["naber", "nasilsin", "gunun nasil"]):
        return "CHITCHAT"

    return "OTHER"


def reply_chitchat(text: str) -> str:
    t = text.lower()

    if "nasılsın" in t:
        return "İyiyim 🙂 Sen nasılsın?"
    if "naber" in t:
        return "Fena değil 😄 Biraz müzik iyi gider mi?"
    if "merhaba" in t or "selam" in t:
        return "Merhaba 👋 Ruh halin nasıl?"
    if "günün nasıl" in t:
        return "Bunu müzikle güzelleştirebiliriz 🎧"

    return "Buradayım 🙂 İstersen ruh halini söyle."


In [12]:
# Kullanıcının yazdığı metni RAG araması için sadeleştirir
# Amaç: embedding modeline daha temiz ve anlamlı bir sorgu vermek

def build_retrieval_query(user_text: str) -> str:

    # 1️⃣ Metni normalize et (küçült, boşlukları düzelt)
    q = user_text.lower()

    # 2️⃣ Gereksiz / anlamsız kelimeleri temizle
    stop_words = [
        "şarkı", "şarkılar", "müzik", "öner", "önerir", "önerir misin",
        "bana", "bir", "birkaç", "lütfen", "istiyorum",
        "misin", "mısın", "acaba"
    ]

    for w in stop_words:
        q = q.replace(w, "")

    q = " ".join(q.split())

    # 3️⃣ Style'a göre arama sorgusunu zenginleştir
    style = infer_style(user_text)

    if style == "odak":
        q += " sakin instrumental odak"
    elif style == "sakin":
        q += " chill yavaş"
    elif style == "huzun":
        q += " melankolik duygusal"
    elif style == "enerji":
        q += " enerjik hareketli"

    return q


In [13]:
# Bu kod, yazım hatalarını tolere eden, duygu durumunu anlayan ve her seferinde daha farklı
# ve tutarlı şarkı önerileri sunan RAG tabanlı bir müzik chatbotu oluşturur.

import re, hashlib, unicodedata
from collections import deque
import random, math


def normalize_query(q: str) -> str:
    q = (q or "").lower()
    q = unicodedata.normalize("NFKC", q)
    q = re.sub(r"[^\w\s]", " ", q)
    q = re.sub(r"\s+", " ", q)
    return q.strip()

_TR_MAP = str.maketrans({
    "ç":"c","ğ":"g","ı":"i","ö":"o","ş":"s","ü":"u",
    "Ç":"c","Ğ":"g","İ":"i","Ö":"o","Ş":"s","Ü":"u",
})

def norm_tr(s: str) -> str:
    s = (s or "").strip().lower().translate(_TR_MAP)
    s = re.sub(r"[^a-z0-9\s\?]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


def _pick(options, seed_text: str) -> str:
    h = int(hashlib.md5(seed_text.encode("utf-8")).hexdigest(), 16)
    return options[h % len(options)]


def infer_style(user_text: str) -> str:
    q = norm_tr(user_text)
    if any(x in q for x in ["ders", "calis", "odak", "sinav"]):
        return "odak"
    if any(x in q for x in ["ayrildim", "ayrilik", "mutsuz", "huzun", "melankolik", "kalbim kirik"]):
        return "huzun"
    if any(x in q for x in ["enerji", "enerjik", "spor", "hareketli"]):
        return "enerji"
    if any(x in q for x in ["gece", "sakin", "chill", "uyku"]):
        return "sakin"
    return "genel"

def make_intro_outro(user_text: str) -> tuple[str, str]:
    style = infer_style(user_text)

    intros = {
        "huzun": [
            "İçini sıkan duyguyu anladım; ağırlaşmadan eşlik edecek bir akış seçtim.",
            "Bu ruh halinde en iyi gelen şey bazen aynı duyguda kalmak; buna uygun bir seçki yaptım.",
            "Kırgınlık ve hüzün baskın; duygunu taşırmadan akacak parçaları topladım."
        ],
        "odak": [
            "Hem kafan dolu hem de odaklanman gerekiyor; dikkat dağıtmadan akacak bir liste yaptım.",
            "Çalışmaya eşlik etsin diye ritmi çok oynatmayan bir akış hazırladım.",
            "Odak moduna uygun, arka planda akacak bir seçki çıkardım."
        ],
        "enerji": [
            "Enerji istiyorsun; temposu düşmeyen bir akış toparladım.",
            "Modu yukarı taşıyacak, hareketli bir seçki hazırladım.",
            "Gazı açacak parçaları bir araya getirdim; akıcı gitsin."
        ],
        "sakin": [
            "Sakin bir mod istiyorsun; yormadan akan bir seçki hazırladım.",
            "Geceye yakışacak, yumuşak bir akış çıkardım.",
            "Sessiz ve dengeli bir mood için uygun parçaları seçtim."
        ],
        "genel": [
            "Söylediğine göre bir akış hazırladım.",
            "Ruh haline göre birkaç parça seçtim.",
            "İstediğin havaya yakın bir akış çıkardım."
        ],
    }

    outros = {
        "huzun": [
            "İstersen daha da “damar” mı olsun yoksa biraz toparlayan mı, söyle ona göre güncelleyeyim.",
            "Daha modern mi, daha arabesk mi istersin? Tarzı yaz, akışı hemen ayarlayayım.",
            "Biraz daha sakinleştireyim mi, yoksa aynı ağırlıkta kalsın mı?"
        ],
        "odak": [
            "Çalışma için daha sakin mi olsun, yoksa hafif tempolu mu? Söyleyince güncellerim.",
            "İstersen söz yoğunluğu az olan bir akışa da çevirebilirim; nasıl istersin?",
            "Daha uzun bir çalışma listesi ister misin? Tarzı yaz yeter."
        ],
        "enerji": [
            "Daha sert mi olsun, yoksa pop/rap karışık mı? Yaz, akışı ona göre keskinleştireyim.",
            "Tempo daha da artsın mı, yoksa böyle akıcı kalsın mı?",
            "İstersen spor moduna özel daha agresif bir liste de çıkarabilirim."
        ],
        "sakin": [
            "İstersen daha da loşlaştırayım mı, yoksa hafif ritim ekleyeyim mi?",
            "Gece sürüşü gibi mi olsun yoksa uyku öncesi gibi mi? Söyle yeter.",
            "Bir tık daha melankolik mi istersin, yoksa daha huzurlu mu?"
        ],
        "genel": [
            "İstersen tarzı (rap/rock/arabesk/pop) veya tempoyu yaz, listeyi ona göre güncelleyeyim.",
            "Tarzı ve tempoyu söylersen daha nokta atışı bir akış çıkarırım.",
            "İstersen daha sakin ya da daha hareketli olacak şekilde revize edebilirim."
        ],
    }

    intro = _pick(intros.get(style, intros["genel"]), user_text + "|intro")
    outro = _pick(outros.get(style, outros["genel"]), user_text + "|outro")
    return intro, outro

RECENT_HISTORY = deque(maxlen=80)

def _weighted_pick_indices(scores, temp=0.7):
    """Score'lara göre ağırlıklı seçim için olasılık üretir."""
    if not scores:
        return []
    mx = max(scores)
    w = [math.exp((s - mx) / max(temp, 1e-6)) for s in scores]
    return w

def pick_diverse(retrieved, rec_n=8, pool_n=40, temp=0.7, max_per_artist=2):
    """
    retrieved formatı: [(score, title, artist), ...]
    - weights=0 hatası VERMEZ
    - tekrarları azaltır (RECENT_HISTORY)
    - yetmezse kademeli olarak kuralları gevşetip rec_n'i doldurur
    """


    cand = []
    seen_local = set()
    for item in retrieved[:min(pool_n, len(retrieved))]:
        if not isinstance(item, (list, tuple)) or len(item) < 3:
            continue
        s, title, artist = item[0], item[1], item[2]
        title = (title or "").strip()
        artist = (artist or "").strip()
        if not title or not artist:
            continue
        key = (title.lower(), artist.lower())
        if key in seen_local:
            continue
        seen_local.add(key)
        try:
            s = float(s)
        except Exception:
            s = 0.0
        cand.append((s, title, artist))

    if not cand:
        return []


    scores = [c[0] for c in cand]
    mx = max(scores)
    base_w = [math.exp((s - mx) / max(temp, 1e-6)) for s in scores]  # > 0

    def _select(ignore_history: bool, per_artist_limit: int):
        picks = []
        used = set()
        artist_count = {}
        weights = base_w[:]


        for _ in range(len(cand) * 4):
            if len(picks) >= rec_n:
                break

            eligible_idx = []
            eligible_w = []

            for i, (s, t, a) in enumerate(cand):
                if weights[i] <= 0:
                    continue
                key = (t.lower(), a.lower())
                if key in used:
                    continue
                if (not ignore_history) and (key in RECENT_HISTORY):
                    continue
                if artist_count.get(a.lower(), 0) >= per_artist_limit:
                    continue

                eligible_idx.append(i)
                eligible_w.append(weights[i])


            if not eligible_idx:
                break


            if sum(eligible_w) <= 0:
                i = random.choice(eligible_idx)
            else:
                i = random.choices(eligible_idx, weights=eligible_w, k=1)[0]

            s, t, a = cand[i]
            weights[i] = 0.0
            key = (t.lower(), a.lower())
            used.add(key)
            artist_count[a.lower()] = artist_count.get(a.lower(), 0) + 1
            picks.append(f"{t} — {a}")

        return picks


    picks = _select(ignore_history=False, per_artist_limit=max_per_artist)


    if len(picks) < rec_n:
        more = _select(ignore_history=True, per_artist_limit=max_per_artist)
        for line in more:
            if line not in picks and len(picks) < rec_n:
                picks.append(line)


    if len(picks) < rec_n:
        more = _select(ignore_history=True, per_artist_limit=999)
        for line in more:
            if line not in picks and len(picks) < rec_n:
                picks.append(line)


    if len(picks) < rec_n:
        for s, t, a in cand:
            line = f"{t} — {a}"
            if line not in picks:
                picks.append(line)
            if len(picks) >= rec_n:
                break

    return picks[:rec_n]




def rag_chat(question: str, top_k: int = 30, rec_n: int = 8, score_threshold: float = 0.03, debug: bool = False):
    q = (question or "").strip()
    if not q:
        print("Bir şey yazarsan sana uygun bir akış çıkarabilirim.")
        return


    q_norm = norm_tr(q)
    if q_norm in ["baska sarkilar oner", "baska sarki oner", "daha fazla oner", "baska oner", "digerlerini oner"]:
        q = q + " ruh halime uygun"


    intent = classify_intent(q)
    if intent == "CHITCHAT":
        print(reply_chitchat(q))
        return
    if intent != "MUSIC":
        print("Bu konuda bilgim yok; ben sadece duygu durumuna göre Türkçe şarkı önerisi yapabiliyorum.")
        return

    rq = build_retrieval_query(q)
    retrieved = retrieve(rq, top_k=top_k)
    if not retrieved:
        print("Bu konuda bilgim yok; ben sadece elimdeki şarkı listesinden öneri yapabiliyorum.")
        return

    best_score = retrieved[0][0]
    if debug:
        print("DEBUG best_score:", best_score)
        print("DEBUG top5:", retrieved[:5])

    if best_score < score_threshold:
        print("Bu konuda bilgim yok; ben sadece elimdeki şarkı listesinden öneri yapabiliyorum.")
        return


    picks = pick_diverse(retrieved, rec_n=rec_n, pool_n=40, temp=0.7, max_per_artist=2)

    if len(picks) < rec_n:
        print("Bu konuda bilgim yok; ben sadece elimdeki şarkı listesinden öneri yapabiliyorum.")
        return


    for line in picks:
        parts = [p.strip() for p in line.split("—")]
        if len(parts) == 2:
            RECENT_HISTORY.append((parts[0].lower(), parts[1].lower()))

    intro, outro = make_intro_outro(q)

    print(intro)
    print("\nÖneriler:\n")
    print("\n".join(picks))
    print("\n" + outro)


In [14]:
# Bu hücre, fine-tuning (LoRA / SFT) sürecinde kullanılan
# gerekli kütüphanelerin güncel sürümlerini kurar.
# Amaç: eğitim sırasında sürüm uyuşmazlığı ve hata almamak.

!pip -q install -U "datasets>=2.20.0" "peft>=0.13.0" "trl>=0.16.0" "accelerate>=0.34.0" "bitsandbytes>=0.45.0"


In [15]:
# Bu hücre, fine-tuning (LoRA / SFT) için eğitim verisi üretir. Amaç: Müzik kategorilerinden otomatik olarak (system, user, assistant) üçlüleri oluşturmak.
# Bu sayede model, verilen context içinden
# doğru formatta şarkı önermeyi öğrenir.
import random
from datasets import Dataset


cats = None
if "music_kb" in globals() and isinstance(music_kb, dict) and "categories" in music_kb:
    cats = music_kb["categories"]
elif "categories" in globals() and isinstance(categories, list):
    cats = categories
else:
    raise ValueError("KB yok. 'categories' veya 'music_kb[\"categories\"]' olmalı.")

random.seed(42)

REC_N = 8
POOL_N = 18
PER_CAT = 10

SYSTEM = (
    "Sen bir 'Mood Music AI' chatbotusun.\n"
    "Kurallar:\n"
    "- Sadece Türkçe.\n"
    "- Kategori/etiket yazma.\n"
    "- Numara yazma.\n"
    "- SADECE Context'teki satırlardan seç.\n"
    "- Çıktı formatı AYNEN şöyle olacak:\n"
    "Öneriler:\n"
    f"<tam olarak {REC_N} satır: Şarkı — Sanatçı>\n"
)

def pick_pool(tracks, n):
    if len(tracks) <= n:
        return tracks[:]
    return random.sample(tracks, n)

def to_line(t):
    return f"{t['title']} — {t['artist']}"

rows = []
for c in cats:
    tracks = c.get("tracks", [])
    tags = c.get("tags", [])
    if len(tracks) < REC_N:
        continue


    tags_str = ", ".join(tags) if isinstance(tags, list) else str(tags)

    for _ in range(PER_CAT):
        pool = pick_pool(tracks, min(POOL_N, len(tracks)))

        picks = random.sample(pool, min(REC_N, len(pool)))

        user = (
            "İstek:\n"
            f"- dil: tr\n"
            f"- öneri_sayısı: {REC_N}\n"
            f"- etiketler: {tags_str}\n"
            "Context:\n" +
            "\n".join(to_line(t) for t in pool)
        )

        assistant = "Öneriler:\n" + "\n".join(to_line(t) for t in picks)

        rows.append({"system": SYSTEM, "user": user, "assistant": assistant})

ds = Dataset.from_list(rows).train_test_split(test_size=0.1, seed=42)
train_ds, val_ds = ds["train"], ds["test"]

print(" Train:", len(train_ds), "| Val:", len(val_ds))
print("\nÖrnek user:\n", train_ds[0]["user"][:400])
print("\nÖrnek assistant:\n", train_ds[0]["assistant"])


 Train: 270 | Val: 30

Örnek user:
 İstek:
- dil: tr
- öneri_sayısı: 8
- etiketler: rock, enerjik, gitar, canlı
Context:
Senden Daha Güzel — Duman
Kufi — Duman
Her Şeyi Yak — Duman
Bir Derdim Var — Mor ve Ötesi
Cambaz — Mor ve Ötesi
Dursun Zaman — maNga
Beni Benimle Bırak — maNga
Holigan — Athena
Senden, Benden, Bizden — Athena
Yalnızlar Rıhtımı — Adamlar
Rüyalarda Buruşmuşuz — Büyük Ev Ablukada
Kendine İyi Bak — Pinhani
Bir Kadın Ç

Örnek assistant:
 Öneriler:
Dursun Zaman — maNga
Rüyalarda Buruşmuşuz — Büyük Ev Ablukada
Kendine İyi Bak — Pinhani
Holigan — Athena
Senden, Benden, Bizden — Athena
Senden Daha Güzel — Duman
Kufi — Duman
Bir Derdim Var — Mor ve Ötesi


In [16]:
# Bu hücre, Mistral modelini LoRA yöntemiyle müzik öneri formatına özel olarak fine-tune eder.
import torch
import inspect
from peft import LoraConfig
from trl import SFTTrainer


try:
    from trl import SFTConfig
    HAS_SFTCONFIG = True
except Exception:
    HAS_SFTCONFIG = False
from transformers import TrainingArguments

def to_text(ex):
    msgs = [
        {"role":"system","content": ex["system"]},
        {"role":"user","content": ex["user"]},
        {"role":"assistant","content": ex["assistant"]},
    ]
    return {"text": tokenizer.apply_chat_template(msgs, tokenize=False)}

train_text = train_ds.map(to_text, remove_columns=train_ds.column_names)
val_text   = val_ds.map(to_text, remove_columns=val_ds.column_names)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)


model.config.use_cache = False

use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
use_fp16 = torch.cuda.is_available() and (not use_bf16)

if HAS_SFTCONFIG:
    args = SFTConfig(
        output_dir="mistral_moodmusic_lora",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        num_train_epochs=2,
        learning_rate=1e-4,
        logging_steps=10,
        save_steps=100,
        max_length=1024,
        packing=True,
        report_to=[],
        bf16=use_bf16,
        fp16=use_fp16,
    )
else:

    args = TrainingArguments(
        output_dir="mistral_moodmusic_lora",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        num_train_epochs=2,
        learning_rate=1e-4,
        logging_steps=10,
        save_steps=100,
        report_to=[],
        bf16=use_bf16,
        fp16=use_fp16,
    )


kw = dict(
    model=model,
    args=args,
    train_dataset=train_text,
    eval_dataset=val_text,
    peft_config=lora_config,
)

sig = inspect.signature(SFTTrainer.__init__)
if "processing_class" in sig.parameters:
    kw["processing_class"] = tokenizer
elif "tokenizer" in sig.parameters:
    kw["tokenizer"] = tokenizer

if "formatting_func" in sig.parameters:
    kw["formatting_func"] = lambda ex: ex["text"]
elif "dataset_text_field" in sig.parameters:
    kw["dataset_text_field"] = "text"

trainer = SFTTrainer(**kw)
trainer.train()
print(" Fine-tuning bitti")


Map:   0%|          | 0/270 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Applying formatting function to train dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss
10,1.319800
20,0.610700


 Fine-tuning bitti


In [17]:
# Bu hücre, LoRA adapter eklemek veya inference yapmak için temel Mistral modelini bellek dostu şekilde yükler.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16
)
print("✅ base yüklendi")


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ base yüklendi


In [18]:
ADAPTER_DIR = "mistral_moodmusic_lora_adapter"
trainer.model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)
print("✅ Adapter kaydedildi ->", ADAPTER_DIR)

from peft import PeftModel
model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()
CLS_MODEL = base
print("✅ Fine-tuned model (adapter) yüklendi.")


✅ Adapter kaydedildi -> mistral_moodmusic_lora_adapter
✅ Fine-tuned model (adapter) yüklendi.


In [19]:
from collections import deque
import math, random

# Son N öneriyi hatırla (uygulama çalıştıkça tekrarları azaltır)
RECENT_HISTORY = deque(maxlen=100)


In [20]:
rag_chat("fenerbahçe birinci olacak mı")
rag_chat("Bu sene enflasyon düşer mi?")
rag_chat("Galatasaray mı Beşiktaş mı daha güçlü?")

# Bu hücrede veriyi negatif sorularla test ettik. Görüldüğü üzere çıktılar gayet başarılı.

Bu konuda bilgim yok; ben sadece duygu durumuna göre Türkçe şarkı önerisi yapabiliyorum.
Bu konuda bilgim yok; ben sadece duygu durumuna göre Türkçe şarkı önerisi yapabiliyorum.
Bu konuda bilgim yok; ben sadece duygu durumuna göre Türkçe şarkı önerisi yapabiliyorum.


In [21]:
rag_chat("Duygusal ve melankolik şarkılar önerir misin?")
rag_chat("Enerjik Türkçe rap şarkılar öner")
rag_chat("Gece dinlemek için sakin ve melankolik Türkçe şarkılar önerir misin?")
 # Bu hücrede verileri kullanarak direkt cevap veren sorularla test ettik ve gayet duygu durumuna göre sonuç verdiğini görebiliyoruz.

İçini sıkan duyguyu anladım; ağırlaşmadan eşlik edecek bir akış seçtim.

Öneriler:

Sakin — Ezhel
Düşünme — Adamlar
Paramparça — Teoman
Mesafe — Semicenk
Kendine İyi Bak — Pinhani
Gurur Benim Neyime — Duman
Yandım — Jakuzi
Araf — maNga

İstersen daha da “damar” mı olsun yoksa biraz toparlayan mı, söyle ona göre güncelleyeyim.
Enerji istiyorsun; temposu düşmeyen bir akış toparladım.

Öneriler:

Nerdesin — Sefo
Kafamın İçi — Motive
Rampapapam — Ben Fero
Affetmem — BLOK3
Beyaz — Motive
Havhavhav — Lvbel C5
Ara — Zeynep Bastık
İlletim — Mehmet Erdem

Tempo daha da artsın mı, yoksa böyle akıcı kalsın mı?
Kırgınlık ve hüzün baskın; duygunu taşırmadan akacak parçaları topladım.

Öneriler:

Dip — Madrigal
Rüzgar — Adamlar
Bir Derdim Var — Mor ve Ötesi
Dua — Gazapizm
Benimle Kayboldun — Pinhani
Geceler — Ezhel
Ne Farkeder — Yüzyüzeyken Konuşuruz
Dudak — Mabel Matiz

Biraz daha sakinleştireyim mi, yoksa aynı ağırlıkta kalsın mı?


In [22]:
rag_chat("Gece ders çalışacağım, kafam dolu, sözlere çok takılmayayım ama tamamen de boş olmasın")
rag_chat("Ne çok yavaş ne çok hareketli, böyle akan ama düşündüren şarkılar olsun")
rag_chat("Yeni ayrıldım, aklım hala onda ama yarın sınavım var, buna göre bir şeyler çalsın")
# Görüldüğü üzere karmaşık ve 2 duygu içeren sorulara rahatlıkla cevap verebiliyor

Hem kafan dolu hem de odaklanman gerekiyor; dikkat dağıtmadan akacak bir liste yaptım.

Öneriler:

Durma Yürüsene — Melike Şahin
Seni Dert Etmeler — Madrigal
Sarmaşık — Dolu Kadehi Ters Tut
Madem — Dolu Kadehi Ters Tut
Unutmak Öyle Kolay mı Sandın — Semicenk
Aldanırım — Derya Bedavacı
Dert Olur — Derya Bedavacı
Bir Kadın Çizeceksin — Manga

Daha uzun bir çalışma listesi ister misin? Tarzı yaz yeter.
Modu yukarı taşıyacak, hareketli bir seçki hazırladım.

Öneriler:

Üzülmedin mi? — Simge
Gülüşünde Bir Şey Var — Yüzyüzeyken Konuşuruz
Yandım — Mabel Matiz
Boş Gemiler — Yüzyüzeyken Konuşuruz
Dursun Zaman — maNga
Yansıma — Hadise
Senden, Benden, Bizden — Athena
Diva Yorgun — Melike Şahin

İstersen spor moduna özel daha agresif bir liste de çıkarabilirim.
Çalışmaya eşlik etsin diye ritmi çok oynatmayan bir akış hazırladım.

Öneriler:

Sen Kaldın — Sıla
Geçme Artık Sokağımdan — Madrigal
Canın Sağ Olsun — Semicenk
İstanbul'da Sonbahar — Teoman
Yoruldum — Sıla
Pişman Değilim — Semicenk
Sende Ka

In [23]:
# Ben ios app çıkardığım için ngrok sitesinden bir token alıp bunu swift kodlarıyla birleştirdim
# ve bir mobil uygulama halinde servis ettim.

!pip -q install fastapi uvicorn pyngrok nest_asyncio

import io, threading, traceback
from contextlib import redirect_stdout
import nest_asyncio
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok


app = FastAPI()

class ChatRequest(BaseModel):
    message: str

class ChatResponse(BaseModel):
    reply: str


RAG_LOCK = threading.Lock()


DEBUG_ERRORS = False

@app.get("/health")
def health():
    return {"ok": True}

def run_rag(message: str) -> str:
    msg = (message or "").strip()
    if not msg:
        return "Bir şey yazarsan sana uygun bir akış çıkarabilirim."


    buf = io.StringIO()

    with RAG_LOCK:
        try:
            with redirect_stdout(buf):
                rag_chat(msg, top_k=12, rec_n=8)

            out = buf.getvalue().strip()


            if not out:
                return "Bir şeyler ters gitti; tekrar dener misin?"
            return out

        except Exception as e:

            print("\n /chat ERROR:", repr(e))
            print(traceback.format_exc())

            if DEBUG_ERRORS:
                return f"Sunucu iç hata: {type(e).__name__}: {e}"
            return "Sunucuda kısa bir hata oldu. Tekrar dener misin?"

@app.post("/chat", response_model=ChatResponse)
def chat(req: ChatRequest):
    reply = run_rag(req.message)
    return {"reply": reply}


nest_asyncio.apply()

def run():

    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning", timeout_keep_alive=30)

threading.Thread(target=run, daemon=True).start()


ngrok.kill()
ngrok.set_auth_token("36WcMBmmkC1EQnY2scN7LwS4Vjq_eyHT4Tt8ovVM5iSepENG")
public_url = ngrok.connect(8000, "http").public_url

print(" NGROK URL:", public_url)
print(" Test et:", public_url + "/health")
print(" Chat:", public_url + "/chat")


 NGROK URL: https://kelsie-hostless-toilfully.ngrok-free.dev
 Test et: https://kelsie-hostless-toilfully.ngrok-free.dev/health
 Chat: https://kelsie-hostless-toilfully.ngrok-free.dev/chat
